In [1]:
from threading import Thread
import socket
import json
import pandas as pd
import copy
import math
import struct
from tqdm import tqdm
import numpy as np
import time

with open('wordseq.txt','r') as f:
        wordlist=eval(f.readline())
data=pd.read_csv('middlesave.txt')
tfidf=np.loadtxt('tfidf.txt')
with open('sim_article_05.txt','r') as f:
        simartdict=eval(f.readline())
with open('synonym.txt','r') as f:
        simworddict=eval(f.readline())

In [2]:
#包装函数来获取函数时间，提高效率时使用的
def get_time_consume(f):
    def inner(*arg, **kwarg):
        s_time = time.time()
        res = f(*arg, **kwarg)
        e_time = time.time()
        print(f.__qualname__, '耗时：{}秒'.format(e_time - s_time))
        return res

    return inner

In [3]:
#some constants
max_return=200#最大返回量常数
maxonetime=0x3fffffff#单次接收最大长度

In [4]:
@get_time_consume#获取函数运行时间用
def hits_articleSort(terms,fuzzy=False):#在朴素排序后，使用hits算法进行优化再返回结果
    print('now enter hits_article')
    expandterms=[]
    if fuzzy==True:
        print('正在进行模糊搜索………………………………')
        for i in terms:
            expandterms.extend(simworddict[i][1:min(4,len(simworddict[i]))])
    else:
        print('正在进行精确搜索………………………………')
    expandterms.extend(terms)
    expandterms=list(set(expandterms))

    naive=naive_articleSort(expandterms)
    print('Now return to hits_articleSort()')
    hadict={}
    artlist=naive
    lennaive=len(naive)
    size=min(200,lennaive)
    maxtimes=200
    minerror=0.0001
    for i in range((size)):
        artlist+=simartdict[naive[i]]
    artlist=list(set(artlist))
    for i in artlist:
        hadict[i]=1
    #hits算法
    error=1000
    for i in range(maxtimes):
        if(error<minerror):
            break
        tmp=copy.deepcopy(hadict)
        norm=0
        for a in artlist:
            score=0
            for j in simartdict[a]:
                if j in tmp.keys() and j is not a:
                    score+=tmp[j]
            hadict[a]=score
            norm+=pow(score,2)
        #标准化
        norm=math.sqrt(norm)
        for a in artlist:
            hadict[a]/=norm
            error=abs(tmp[a]-hadict[a])
    #按照authority排序
    hits=sorted(artlist,key=lambda ind:hadict[ind],reverse=True)
    return hits[:min(max_return,len(hits))]


In [5]:
@get_time_consume
def naive_articleSort(terms):#朴素排序，将tfidf中单词对应值加和，然后按照这个分数从大到小排序
    print('Now enter naive_articleSort()')
    respond=[]
    articlenum=len(data.body)
    score=[0 for i in range(articlenum)]
    for keyword in terms:
        if keyword not in wordlist:
            continue
        index=wordlist.index(keyword)
        for i in range(articlenum):
            #score[i]+=eval(data['tf-idf'][i])[index]
            score[i]+=tfidf[i][index]
    score_index=list(zip(score,range(len(score))))
    score_index.sort(key=lambda t:t[0],reverse=True)
    for i in tqdm(range(max_return)):
        if score_index[i][0]==0:#0了，说明后面的单词都未出现过，结束计算
            break
        ind=score_index[i][1]
        respond.append(ind)
    return respond

def func(m,fuzzy,conn):
    
    print('Now enter func()')
    ans=[(data.title[ind],data.content[ind]) for ind in hits_articleSort(eval(str(m)),fuzzy)]
    with open('answer.txt','w') as fi:
        fi.write(str(ans))
    sendans=str(ans).encode('utf-8')
    
    sendtimes=len(ans)
    print('sendtimes=',sendtimes)
    conn.send(str(sendtimes).encode('utf-8'))
    for i in range(sendtimes):
        print('------------------------------------------现在发送第',i+1,'个：')
        btitle=ans[i][0].encode('utf-8')
        bcont=ans[i][1].encode('utf-8')
        #lentitle=str(len(btitle)).zfill(10)
        lentitle=struct.pack('i', len(btitle))
        conn.send(lentitle) 
        print('已发送标题大小',end=' ')  
        conn.send(btitle)
        print('已发送标题',end=' ')  
        #lencon=str(len(bcont)).zfill(10)   
        lencon=struct.pack('i', len(bcont))
        conn.send(lencon) 
        print('已发送cont大小',end=' ') 
        conn.send(bcont) 
        print('已发送cont') 

class LocalServer(object):
    def __init__(self, host, port):
        self.address = (host, port)

    def run(self):
        print('初始化服务器主机套接字对象……')
        server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        print('绑定的主机信息……')
        server.bind(self.address)
        print('开始启动服务器……')
        server.listen(5)
        
        """
        TODO：请在服务器端实现合理的并发处理方案，使得服务器端能够处理多个客户端发来的请求
        """
    
        """
        TODO: 请补充实现文本检索，以及服务器端与客户端之间的通信
        
        1. 接受客户端传递的数据， 例如检索词
        2. 调用检索函数，根据检索词完成检索
        3. 将检索结果发送给客户端，具体的数据格式可以自己定义
        
        """
        while True:
            print('等待客户端链接……')
            conn,addr=server.accept()
            print('链接的客户端套接字对象:{}\n客户端IP地址为:{}'.format(conn,addr))

            getfuzzyornot = conn.recv(1)
            fuzzyornot = struct.unpack('?', getfuzzyornot)[0]
            print('收到fuzzyornot=',fuzzyornot,type(fuzzyornot),end=' ')

            gettermslen = conn.recv(4)
            btermslen = struct.unpack('i', gettermslen)[0]
            m=conn.recv(btermslen).decode('utf-8')
            t=Thread(target=func,args=(m,fuzzyornot,conn))
            t.start()
            
        server.close()
        
    
        

#### 运行服务器端
启动服务器之后，在run.ipynb中运行客户端图形界面

In [6]:
print('初始化服务器主机信息……')
server = LocalServer("0.0.0.0", 1234)
if __name__=='__main__':
    server.run()

初始化服务器主机信息……
初始化服务器主机套接字对象……
绑定的主机信息……
开始启动服务器……
等待客户端链接……


100%|██████████| 200/200 [00:00<00:00, 353949.70it/s]

链接的客户端套接字对象:<socket.socket fd=59, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 1234), raddr=('127.0.0.1', 64438)>
客户端IP地址为:('127.0.0.1', 64438)
收到fuzzyornot= True <class 'bool'> Now enter func()
now enter hits_article
正在进行模糊搜索………………………………等待客户端链接……

Now enter naive_articleSort()
naive_articleSort 耗时：0.03691291809082031秒
Now return to hits_articleSort()
hits_articleSort 耗时：0.050804853439331055秒
sendtimes= 200
------------------------------------------现在发送第 1 个：
已发送标题大小 已发送标题 已发送cont大小 已发送cont
------------------------------------------现在发送第 2 个：
已发送标题大小 已发送标题 已发送cont大小 已发送cont
------------------------------------------现在发送第 3 个：
已发送标题大小 已发送标题 已发送cont大小 已发送cont
------------------------------------------现在发送第 4 个：
已发送标题大小 已发送标题 已发送cont大小 已发送cont
------------------------------------------现在发送第 5 个：
已发送标题大小 已发送标题 已发送cont大小 已发送cont
------------------------------------------现在发送第 6 个：
已发送标题大小 已发送标题 已发送cont大小 已发送cont
------------------------------------

 已发送cont大小 已发送cont
------------------------------------------现在发送第 122 个：
已发送标题大小 已发送标题 已发送cont大小 已发送cont
------------------------------------------现在发送第 123 个：
已发送标题大小 已发送标题 已发送cont大小 已发送cont
------------------------------------------现在发送第 124 个：
已发送标题大小 已发送标题 已发送cont大小 已发送cont
------------------------------------------现在发送第 125 个：
已发送标题大小 已发送标题 已发送cont大小 已发送cont
------------------------------------------现在发送第 126 个：
已发送标题大小 已发送标题 已发送cont大小 已发送cont
------------------------------------------现在发送第 127 个：
已发送标题大小 已发送标题 已发送cont大小 已发送cont
------------------------------------------现在发送第 128 个：
已发送标题大小 已发送标题 已发送cont大小 已发送cont
------------------------------------------现在发送第 129 个：
已发送标题大小 已发送标题 已发送cont大小 已发送cont
------------------------------------------现在发送第 130 个：
已发送标题大小 已发送标题 已发送cont大小 已发送cont
------------------------------------------现在发送第 131 个：
已发送标题大小 已发送标题 已发送cont大小 已发送cont
------------------------------------------现在发送第 132 个：
已发送标题大小 已发送标题 已发送cont大小 已发送cont
------------------------